# Modeling the phase shift due to thermal conduction in the sensor

We modelize the phase shift with a second order low-pass filter. In order to determine the filter parameters, use the following functions.

You need the measurement of a temperature step response.

In [ ]:
import numpy as np
from scipy.signal import lti, lsim
from scipy.optimize import minimize

In [ ]:
# Determines the second-order filter corresponding to the data.
# Uses the measurement of a temperature step response
# T_final: final temperature reached (setpoint)
# T_measured: array of measured temperatures
# time: array of times corresponding to the measurements
# Returns the optimized parameters tau1 and tau2 of the second-order filter (this filter has gain 1)
def find_parameters(T_final, T_measured, time):
    T_initial = T_measured[0]

    # Search for the time constant
    i_tau = 0
    while T_measured[i_tau] < T_initial + (T_final - T_initial) * 0.63:
        i_tau += 1
    tau = time[i_tau]

    # 5% response time
    i_tr5 = 0
    while T_measured[i_tr5] < T_initial + (T_final - T_initial) * 0.95:
        i_tr5 += 1
    tr5 = time[i_tr5]

    # Optimization of tau1 and tau2 to minimize the squared error
    # between the model response and the measurement
    def distance(tau_vector):
        tau1 = tau_vector[0]
        tau2 = tau_vector[1]
        ft = lti([1], [tau1 * tau2, tau1 + tau2, 1])
        t_ft, y_ft, _ = lsim(ft, U=df['Tc'].iloc[:t1] - T_initial, T=time)
        y_ft = y_ft + T_initial
        return np.sum((y_ft - T_measured) ** 2)

    opt_result = minimize(distance, x0=[tau / 10, tr5 / 3])
    tau1_opt, tau2_opt = opt_result.x
    return tau1_opt, tau2_opt

# Computes the response of the second-order filter
# tau1, tau2: filter parameters
def response(tau1, tau2, T_setpoint, T_initial, time):
    ft = lti([1], [tau1 * tau2, tau1 + tau2, 1])
    t_resp, y_resp, _ = lsim(ft, U=T_setpoint - T_initial, T=time)  # Response to the setpoint
    y_resp = y_resp + T_initial
    return t_resp, y_resp

# Computes the mean squared error between the model response and the measured data
def accuracy(tau1, tau2, T_setpoint, T_initial, time, T_measured):
    ft = lti([1], [tau1 * tau2, tau1 + tau2, 1])
    t_resp, y_resp, _ = lsim(ft, U=T_setpoint - T_initial, T=time)  # Response to the setpoint
    y_resp = y_resp + T_initial
    MSE = np.sum((y_resp - T_measured) ** 2) / len(T_measured)
    return MSE